In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 10.3 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

<h1>Hepler Functions</h1>


In [ ]:
def resize_video(input_path, output_path, target_height=1280):
    """
    Resize a video while retaining its aspect ratio and save as a new MP4 file.

    Parameters:
    - input_path: Path to the input video file.
    - output_path: Path to save the resized video file.
    - target_height: Desired height of the output video.
    """

    # Open the input video file
    cap = cv2.VideoCapture(input_path)

    # Get the original video dimensions
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate the new width while maintaining aspect ratio
    aspect_ratio = original_width / original_height
    new_width = int(target_height * aspect_ratio)

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, target_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame while maintaining aspect ratio
        resized_frame = cv2.resize(frame, (new_width, target_height))

        # Write the resized frame to the output video
        out.write(resized_frame)

    # Release the video objects
    cap.release()
    out.release()

def get_video_details(video_path):
    """
    Retrieve width, height, and number of pixels of a video.

    Parameters:
    - video_path: Path to the video file.

    Returns:
    - tuple: (width, height, number of pixels)
    """

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the width and height of the video frames
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate the number of pixels in a single frame
    pixels = width * height

    # Release the video file
    cap.release()

    print(f"Video Width: {width}")
    print(f"Video Height: {height}")
    print(f"Number of Pixels per Frame: {pixels}")

In [ ]:
# model = YOLO('yolov8x-pose-p6.pt')

# # results = model(source=VIDEO_PATH_CROPPED, show=True, conf=0.3, verbose=True)
# results = model(source=VIDEO_PATH_CROPPED, conf=0.3, save = True)

In [ ]:
import cv2
from ultralytics import YOLO

VIDEO_PATH = '/content/drive/Shareddrives/TAMU Datathon 2023/dataset/videos/harsh-v2-fail.mp4'
VIDEO_PATH_CROPPED ='cropped.mp4'
resize_video(VIDEO_PATH, VIDEO_PATH_CROPPED, 1280)
# Load a model
model = YOLO('yolov8x-pose-p6.pt')  # load an official model

# Process each frame
import cv2

def split_video_into_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frameNum = 5
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if no more frames are available

        # Append the frame to the list
        if(frameNum == 0):
          frames.append(frame)
          frameNum = 3
        else:
          frameNum -= 1

    # Release the video capture object
    cap.release()

    return frames

# Example usage:
frames_list = split_video_into_frames(VIDEO_PATH_CROPPED)

# print(frames_list)
# Now, frames_list contains all the frames from the video
# Run batched inference on a list of images
results = model(frames_list, conf=0.3)  # return a list of Results objects

boxes = []
masks = []
keypoints = []
probs = []

# Process results list
for result in results:
    boxes.append(result.boxes)  # Boxes object for bbox outputs
    masks.append(result.masks)  # Masks object for segmentation masks outputs
    keypoints.append(result.keypoints)  # Keypoints object for pose outputs
    probs.append(result.probs)  # Probs object for classification outputs



# for frame, keypoints in zip(frames_list, keypoints):
#     # Draw keypoints on the frame
#     for kp in keypoints:
#       print(kp)
#       x, y, _, _ = map(int, kp)
#       cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Draw a green circle at each keypoint

#     # Display the frame with keypoints
#     cv2.imshow('Frame with Keypoints', frame)
#     cv2.waitKey(0)

# cv2.destroyAllWindows()

100%|██████████| 190M/190M [00:00<00:00, 283MB/s]



OutOfMemoryError: ignored

In [ ]:
import pickle

In [ ]:
framePoints = []

for frame in range(len(keypoints)):
  # print(keypoints[frame])
  # plt.figure(figsize = (4,4))
  confs = keypoints[frame].conf
  if(confs is not None):
    confIndexLis = []
    for i, conf in enumerate(confs[0]):
      if conf > 0.0:
        confIndexLis.append(i)
    # print(confIndexLis)

    xys = keypoints[frame].xy[0].tolist()
    # print(xys)
    points = []
    for idx in confIndexLis:
      points.append(xys[idx])

    print(points)
    framePoints.append(points)

with open('output_harshFail.pkl', 'wb') as f:
  pickle.dump(framePoints, f)

    # plt.scatter(x, y, color="red", marker='o', s=20)

    # plt.xlabel('X')
    # plt.ylabel('Y')
    # plt.title('Skeleton for frame ' + str(frame))
    # plt.show()

In [ ]:
from ultralytics.utils.plotting import Annotator
import cv2
import numpy as np

print(framePoints[20])

img = np.zeros([1280,720,3],dtype=np.uint8)
img.fill(255)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_harshFail.mp4', fourcc, 20.0, (720, 1280))

for pointsArr in framePoints:
  img.fill(255)
  annotator = Annotator(img)
  annotator.kpts(np.array(pointsArr))

  out.write(annotator.result())

out.release()
print('done')